In [22]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

# Set the random seed for reproducibility
g = torch.Generator().manual_seed(2147483647)

In [23]:
# Load the raw text data
words = open('names.txt', 'r').read().splitlines()

# Construct the vocabulary
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos) # 27

# Hyperparameters
block_size = 3 # Context length: predict the 4th character from 3 previous

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # Slide the window
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

# Split the dataset: 80% Train, 10% Dev, 10% Test
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [24]:
n_embd = 10  # Embedding dimensionality
n_hidden = 64 # Hidden layer neurons

# 1. Embedding Matrix
C = torch.randn((vocab_size, n_embd), generator=g)

# 2. Layer 1 Weights (Kaiming Init)
# We multiply by (5/3) / sqrt(fan_in) where fan_in is n_embd * block_size (30)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
# Layer 1 Bias (scaled down to reduce initial impact)
b1 = torch.randn(n_hidden, generator=g) * 0.1

# 3. Layer 2 Weights (Output Projection)
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# 4. Batch Normalization Parameters
# Scale (gamma) initialized to 1.0 + small noise (not 0!)
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
# Shift (beta) initialized to 0 + small noise
bnbias = torch.randn((1, n_hidden))*0.1

# Collect all parameters
parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

print(f"Total Parameters: {sum(p.nelement() for p in parameters)}")

Total Parameters: 4137


In [25]:
# Construct a minibatch of size 32
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

# --- Layer 1: Embedding ---
emb = C[Xb] # (32, 3, 10)
embcat = emb.view(-1, block_size * n_embd) # Flatten to (32, 30)

# --- Layer 2: Linear Projection ---
# Matrix multiplication: (32, 30) @ (30, 64) + (64,) -> (32, 64)
hprebn = embcat @ W1 + b1 

# --- Layer 3: Batch Normalization (Decomposed) ---
# 1. Calculate Mean across batch (dim 0)
bnmeani = 1/n * hprebn.sum(0, keepdim=True)
# 2. Calculate Centered Data
bndiff = hprebn - bnmeani
# 3. Calculate Squared Differences
bndiff2 = bndiff**2
# 4. Calculate Variance (Bessel's Correction applied: n-1)
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True)
# 5. Calculate Inverse Standard Deviation (adding epsilon for stability)
bnvar_inv = (bnvar + 1e-5)**-0.5
# 6. Normalize
bnraw = bndiff * bnvar_inv
# 7. Scale and Shift (Gamma and Beta)
hpreact = bngain * bnraw + bnbias

# --- Layer 4: Non-linearity ---
h = torch.tanh(hpreact) # (32, 64)

# --- Layer 5: Linear Projection to Logits ---
logits = h @ W2 + b2 # (32, 64) @ (64, 27) + (27,) -> (32, 27)

# --- Layer 6: Cross Entropy Loss (Decomposed) ---
# 1. Numerical Stability (Max trick)
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
# 2. Exponentiate
counts = norm_logits.exp()
# 3. Sum counts for normalization
counts_sum = counts.sum(1, keepdim=True)
# 4. Invert sum
counts_sum_inv = counts_sum**-1
# 5. Probabilities (Softmax)
probs = counts * counts_sum_inv
# 6. Log Probabilities
logprobs = probs.log()
# 7. Negative Log Likelihood Loss
# Index into the rows (range(n)) at the columns specified by targets (Yb)
loss = -logprobs[range(n), Yb].mean()

print(f"Loss: {loss.item()}")

Loss: 3.346653461456299


In [ ]:
# Reset gradients
for p in parameters:
    p.grad = None

# We must explicitly call retain_grad() on intermediate tensors
# because PyTorch normally discards non-leaf gradients to save memory.
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, 
          norm_logits, logits, h, hpreact, bnraw, bnvar_inv, 
          bnvar, bndiff2, bndiff, bnmeani, hprebn, embcat, emb]:
    t.retain_grad()

# Backward pass
loss.backward()

In [ ]:
def cmp(s, dt, t):
    if t.grad is None:
        print(f'{s:15s} | exact: False | approximate: False | maxdiff: inf (grad is None)')
        return
    # Ensure shapes match
    if dt.shape != t.grad.shape:
        print(f'{s:15s} | exact: False | approximate: False | maxdiff: inf (shape mismatch: {dt.shape} vs {t.grad.shape})')
        return
    # Compare tensors
    diff = (dt - t.grad)
    ex = torch.all(diff == 0).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = diff.abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [28]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


In [29]:
dprobs = (1.0 / probs) * dlogprobs
cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


In [30]:
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


In [31]:
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


In [32]:
dcounts = counts_sum_inv * dprobs # Contribution from probs
dcounts += torch.ones_like(counts) * dcounts_sum # Contribution from sum
cmp('counts', dcounts, counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0


In [33]:
dnorm_logits = counts * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0


In [34]:
dlogits = dnorm_logits.clone() # Path 1
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes # Path 2
cmp('logits', dlogits, logits)

logits          | exact: True  | approximate: True  | maxdiff: 0.0


In [35]:
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)

h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0


In [36]:
dhpreact = (1.0 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10


In [37]:
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnbias = dhpreact.sum(0, keepdim=True)
dbnraw = bngain * dhpreact

cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)

bngain          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
bnbias          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
bnraw           | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10


In [38]:
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw # Only the first contribution
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

bnvar_inv       | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09


In [39]:
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

bnvar           | exact: False | approximate: True  | maxdiff: 6.984919309616089e-10


In [40]:
dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

bndiff2         | exact: False | approximate: True  | maxdiff: 2.1827872842550278e-11


In [41]:
dbndiff += (2 * bndiff) * dbndiff2
cmp('bndiff', dbndiff, bndiff)

bndiff          | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10


In [42]:
dbnmeani = (-dbndiff).sum(0, keepdim=True)
dhprebn = dbndiff.clone() # Path 1 contribution
cmp('bnmeani', dbnmeani, bnmeani)

/tmp/ipykernel_26770/614943589.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  ex = torch.all(dt == t.grad).item()


TypeError: all() received an invalid combination of arguments - got (bool), but expected one of:
 * (Tensor input, *, Tensor out = None)
 * (Tensor input, tuple of ints dim = None, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, int dim, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, name dim, bool keepdim = False, *, Tensor out = None)


In [43]:
dhprebn += (1.0/n) * (torch.ones_like(hprebn) * dbnmeani)
cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10


In [44]:
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

cmp('W1', dW1, W1)
cmp('b1', db1, b1)

W1              | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
b1              | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09


In [45]:
demb = dembcat.view(emb.shape[0], emb.shape[1], emb.shape[2])
cmp('emb', demb, emb)

emb             | exact: False | approximate: True  | maxdiff: 1.3969838619232178e-09


In [46]:
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):   # Iterate over batch
    for j in range(Xb.shape[1]): # Iterate over context position
        ix = Xb[k,j]
        dC[ix] += demb[k,j] # Accumulate gradient

cmp('C', dC, C)

C               | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09


In [47]:
# Fused Softmax gradient
# The gradient of cross-entropy loss w.r.t. logits is: (probs - one_hot(Yb)) / n
probs_fast = F.softmax(logits, dim=1)
dlogits_fast = probs_fast.clone()
# Subtract 1 from the correct class positions
dlogits_fast[range(n), Yb] -= 1.0
# Divide by batch size
dlogits_fast /= n

cmp('logits_fast', dlogits_fast, logits)

logits_fast     | exact: False | approximate: True  | maxdiff: 4.6566128730773926e-09


In [ ]:
# Fused Batch Norm gradient
# This combines all the paths: through bnraw, through mean, and through variance
bn_var_inv = (bnvar + 1e-5)**-0.5
aux = (dlogits @ W2.T) * (1.0 - h**2) # This is dhpreact
# Multiply by bngain to get dbnraw (from Cell 15: dbnraw = bngain * dhpreact)
dbnraw_fast = bngain * aux
# The fused formula accounts for all contributions
# Note: variance uses (n-1) for Bessel's correction, so we need to account for that
dhprebn_fast = bn_var_inv * (
    dbnraw_fast 
    - dbnraw_fast.sum(0, keepdim=True) / n  # Mean contribution
    - bnraw * (dbnraw_fast * bnraw).sum(0, keepdim=True) / (n - 1)  # Variance contribution (uses n-1)
)

cmp('hprebn_fast', dhprebn_fast, hprebn)

hprebn_fast     | exact: False | approximate: False | maxdiff: 3.117695450782776e-05
